###Building ChatBot with Message History

In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
# Initialize the Groq model
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EC1C58AE30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EC1C589E70>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Nilesh and I am a chef AI Engineer")])

AIMessage(content="That's a fascinating combination!  I can see how your skills as a chef and an AI engineer could be a powerful duo. \n\nWhat kind of projects are you working on? Are you developing AI systems to help chefs in the kitchen, like recipe recommendations or automated cooking assistants? Or are you focusing on using AI to analyze food trends and create innovative dishes? \n\nI'd love to hear more about your work!  \n\n", response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 23, 'total_tokens': 115, 'completion_time': 0.167272727, 'prompt_time': 0.000140089, 'queue_time': 0.01331685, 'total_time': 0.167412816}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-60b61b45-a6d8-4c22-96f5-c44515974557-0', usage_metadata={'input_tokens': 23, 'output_tokens': 92, 'total_tokens': 115})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Nilesh and I am a chef AI Engineer"),
        AIMessage(content="That's a fascinating combination!  I can see how your skills as a chef and an AI engineer could be a powerful duo.  \n\nWhat kind of projects are you working on? Are you developing AI systems to help chefs in the kitchen, like recipe recommendations or automated cooking assistants? Or are you focusing on using AI to analyze food trends and create innovative dishes? \n\nI'd love to hear more about your work!  \n\n"),
        HumanMessage(content="Hey Whats my name and what do I do?")
    ]
)


AIMessage(content="You are Nilesh, a chef AI Engineer!  \n\nRemember, I'm still learning and getting to know you.  Is there anything else you'd like me to remember about you? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 132, 'total_tokens': 177, 'completion_time': 0.081818182, 'prompt_time': 0.005052644, 'queue_time': 0.008276565, 'total_time': 0.086870826}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8369f22e-1257-4105-8c21-7656d9023e10-0', usage_metadata={'input_tokens': 132, 'output_tokens': 45, 'total_tokens': 177})

###Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [11]:
config = {"configurable": {"session_id":"chat1"}}

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Nilesh and I am a chef AI Engineer")],
    config=config
)

In [14]:
response.content

"That's a really cool name!  \n\nIt sounds like you have a unique perspective, blending the creativity of a chef with the analytical power of AI.  \n\nWhat kind of projects are you working on? I'm curious to know how you're using AI to enhance the culinary world. \n\n"

In [15]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Nilesh!  \n\nYou told me at the beginning of our conversation.  😊  \n\n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 215, 'total_tokens': 250, 'completion_time': 0.063636364, 'prompt_time': 0.008291156, 'queue_time': 0.005182844000000001, 'total_time': 0.07192752}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-556b0784-cdc2-4181-b61b-4554ecd4dd2e-0', usage_metadata={'input_tokens': 215, 'output_tokens': 35, 'total_tokens': 250})

In [22]:
## Change the config-->session id
config1 = {"configurable": {"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it!\n"

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="my name is Azar")],
    config=config1
)
response.content

"It's nice to meet you, Azar!  😊 \n\nIs there anything I can help you with today? \n"

In [24]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

'Your name is Azar.  \n\nI remember!  Do you have any other questions for me? 😊 \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [25]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [27]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Nilesh")]})

AIMessage(content="Hello Nilesh! It's nice to meet you. What can I do for you today?  \n", response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 31, 'total_tokens': 55, 'completion_time': 0.043636364, 'prompt_time': 0.000523497, 'queue_time': 0.013840553, 'total_time': 0.044159861}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8077153a-1679-487e-a6e0-12ee227bf32c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 24, 'total_tokens': 55})

In [28]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [29]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Nilesh")],
    config=config
)

response

AIMessage(content="It's nice to meet you, Nilesh! \n\nHow can I help you today? 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 31, 'total_tokens': 57, 'completion_time': 0.047272727, 'prompt_time': 0.000432628, 'queue_time': 0.015047172000000001, 'total_time': 0.047705355}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9d4ea147-91ea-4c73-9348-a46f36b6f9f6-0', usage_metadata={'input_tokens': 31, 'output_tokens': 26, 'total_tokens': 57})

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Nilesh. 😊  \n\nI remember!  How can I help you further? \n'

In [31]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [33]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Neelesh")],"language":"Hindi"})
response.content

'नमस्ते नीलेश!  मुझे आपसे मिलकर अच्छा लगा।  \n\nक्या मैं आपकी कोई मदद कर सकता हूँ? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [34]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [36]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Neelesh")],"language":"Telugu"},
    config=config
)
repsonse.content

'నమస్కారం, నీలేశ్! 😊 \n\nనేను మీకు సహాయం చేయడానికి ఇక్కడ ఉన్నాను. \n\nమీరు ఏమి అడిగితే అనండి. 👍\n'

In [37]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Telugu"},
    config=config,
)
response.content

'మీ పేరు నీలేశ్. 😊 \n\n\nమీరు తిరిగి అనవలసిన అవసరం లేదు.  😉\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [38]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\LangChain2\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\LangChain2\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\E.NILESH\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.mi

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [39]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(message=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What ice cream do i like")],
    "language": "English"
    }
)
response.content

'You said you like vanilla ice cream!  🍦 😊 \n'

In [40]:
response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="what math problem did i ask")],
    "language": "English"
    }
)
response.content

'You asked "what\'s 2 + 2" 😊 \n\n\n'

In [41]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [42]:
response = with_message_history.invoke(
    {
    "messages": messages + [HumanMessage(content="whats my name?")],
    "language": "English"
    },
    config=config
)
response.content

'Your name is Bob!  \n\nI remember that from our earlier conversation. 😊\n'